In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import os
import re
import requests
from bs4 import BeautifulSoup

# API calls

* We will play around with the velib API
* velib is a bike sharing service in Paris

* `The task is to create a function that returns the closest velib station to a given address by making a call to the API`

* We will use the requests library to make the call to the API
* We will use the json library to parse the response

* We will use the geopy library to get the coordinates of the address
* We will use the geopy library to calculate the distance between the address and the velib stations

* We will use the folium library to plot the velib stations on a map


In [3]:

api = 'https://opendata.paris.fr/api/records/1.0/search/?dataset=velib-disponibilite-en-temps-reel&q=&lang=en&facet=name&facet=is_installed&facet=is_renting&facet=is_returning&facet=nom_arrondissement_communes&timezone=Africa%2FNairobi'

# 1: call the API and parse the response
# ie
response = requests.get(api)
response_json = response.json()
response_json

{'nhits': 1464,
 'parameters': {'dataset': 'velib-disponibilite-en-temps-reel',
  'lang': 'en',
  'rows': 10,
  'start': 0,
  'facet': ['name',
   'is_installed',
   'is_renting',
   'is_returning',
   'nom_arrondissement_communes'],
  'format': 'json',
  'timezone': 'Africa/Nairobi'},
 'records': [{'datasetid': 'velib-disponibilite-en-temps-reel',
   'recordid': '510b96609c43f3981aec7d7739c5cf9b9fe30661',
   'fields': {'name': "Rouget de L'isle - Watteau",
    'stationcode': '44015',
    'ebike': 0,
    'mechanical': 0,
    'coordonnees_geo': [48.778192750803, 2.3963020229163],
    'duedate': '2023-02-09T12:35:37+03:00',
    'numbikesavailable': 0,
    'numdocksavailable': 0,
    'capacity': 0,
    'is_renting': 'NON',
    'is_installed': 'NON',
    'nom_arrondissement_communes': 'Vitry-sur-Seine',
    'is_returning': 'NON'},
   'geometry': {'type': 'Point',
    'coordinates': [2.3963020229163, 48.778192750803]},
   'record_timestamp': '2023-06-09T04:54:00.417+03:00'},
  {'datasetid':

In [5]:
# 2: get the coordinates of the address
# ie
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_app")
location = geolocator.geocode("175 5th Avenue NYC")
location.latitude, location.longitude

(40.741059199999995, -73.98964162240998)

In [6]:
# 3: calculate the distance between the address and the velib stations
# ie
from geopy.distance import geodesic
geodesic((location.latitude, location.longitude), (48.8566, 2.3522)).km


5849.964745570385

In [8]:
# 4: plot the velib stations on a map
# ie
import folium
m = folium.Map(location=[48.8566, 2.3522], zoom_start=12)
folium.Marker([48.8566, 2.3522], popup='Paris').add_to(m)
m

In [15]:
# 5: create a function that returns the closest velib station to a given address and plot it on a map
# the function should take the address as an argument and return the name of the closest velib station
# It should return map, distance, name of the closest velib station
# ie
def get_closest_velib_station(address):
    # call the API and parse the response
    response = requests.get(api)
    response_json = response.json()
    # get the coordinates of the address
    geolocator = Nominatim(user_agent="my_app")
    location = geolocator.geocode(address)
    # calculate the distance between the address and the velib stations
    distances = []
    for record in response_json['records']:
        lat = record['geometry']['coordinates'][0]
        lon = record['geometry']['coordinates'][1]
        distance = geodesic((location.latitude, location.longitude), (lat, lon)).km
        distances.append(distance)
    # get the name of the closest velib station
    closest_station_index = np.argmin(distances)
    closest_station_name = response_json['records'][closest_station_index]['fields']['name']
    # plot the velib stations on a map
    m = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)
    folium.Marker([location.latitude, location.longitude], popup=address).add_to(m)
    folium.Marker([response_json['records'][closest_station_index]['geometry']['coordinates'][1], response_json['records'][closest_station_index]['geometry']['coordinates'][0]], popup=closest_station_name).add_to(m)
    return m, distances[closest_station_index], closest_station_name

get_closest_velib_station("175 5th Avenue NYC")

(<folium.folium.Map at 0x1a3ff680be0>,
 12516.716039702485,
 'Bois de Vincennes - Gare')

In [16]:
m

## Part 2: Data Analysis
### Next step: analyse the data
* ie
* 1: get the number of velib stations in each arrondissement
*  2: get the number of velib stations in each arrondissement that are currently available
*  3: get the number of velib stations in each arrondissement that are currently unavailable
*  4: get the number of velib stations in each arrondissement that are currently unavailable and have no bikes available
* 5: get the number of velib stations in each arrondissement that are currently unavailable and have no parking slots available
* 6: get the number of velib stations in each arrondissement that are currently unavailable and have no bikes available and have no parking slots available
* 7: get the number of velib stations in each arrondissement that are currently unavailable and have no bikes available and have no parking slots available and have no station status available
* 8: get the number of velib stations in each arrondissement that are currently unavailable and have no bikes available and have no parking slots available and have no station status available and have no station code available

In [17]:
# 1: get the number of velib stations in each arrondissement
# ie
arrondissements = []
for record in response_json['records']:
    arrondissement = record['fields']['nom_arrondissement_communes']
    arrondissements.append(arrondissement)

arrondissements = pd.Series(arrondissements)
arrondissements.value_counts()


Paris                   3
Vitry-sur-Seine         1
Saint-Denis             1
Bobigny                 1
Aubervilliers           1
Ivry-sur-Seine          1
Boulogne-Billancourt    1
Rueil-Malmaison         1
dtype: int64

In [18]:
# 2: get the number of velib stations in each arrondissement that are currently available
# ie
arrondissements = []
for record in response_json['records']:
    if record['fields']['is_renting'] == 'OUI':
        arrondissement = record['fields']['nom_arrondissement_communes']
        arrondissements.append(arrondissement)

arrondissements = pd.Series(arrondissements)
arrondissements.value_counts()

Paris                   3
Saint-Denis             1
Aubervilliers           1
Ivry-sur-Seine          1
Boulogne-Billancourt    1
Rueil-Malmaison         1
dtype: int64

In [19]:
# 3: get the number of velib stations in each arrondissement that are currently unavailable
# ie
arrondissements = []
for record in response_json['records']:
    if record['fields']['is_renting'] == 'NON':
        arrondissement = record['fields']['nom_arrondissement_communes']
        arrondissements.append(arrondissement)

arrondissements = pd.Series(arrondissements)
arrondissements.value_counts()

Vitry-sur-Seine    1
Bobigny            1
dtype: int64

In [20]:
# 4: get the number of velib stations in each arrondissement that are currently unavailable and have no bikes available
# ie
arrondissements = []
for record in response_json['records']:
    if record['fields']['is_renting'] == 'NON' and record['fields']['numbikesavailable'] == 0:
        arrondissement = record['fields']['nom_arrondissement_communes']
        arrondissements.append(arrondissement)

arrondissements = pd.Series(arrondissements)
arrondissements.value_counts()

Vitry-sur-Seine    1
Bobigny            1
dtype: int64

In [21]:
# 5: get the number of velib stations in each arrondissement that are currently unavailable and have no parking slots available
# ie
arrondissements = []
for record in response_json['records']:
    if record['fields']['is_renting'] == 'NON' and record['fields']['numdocksavailable'] == 0:
        arrondissement = record['fields']['nom_arrondissement_communes']
        arrondissements.append(arrondissement)

arrondissements = pd.Series(arrondissements)
arrondissements.value_counts()

Vitry-sur-Seine    1
Bobigny            1
dtype: int64